In [1]:
print('Importing packages...')
import os
#import urllib
import pandas
#import zipfile
import time
import numpy as np
from matplotlib import pyplot as plt
import geopandas
import re
from IPython.display import display, HTML

#census_data_dir = '/home/idies/workspace/Temporary/raddick/cra_scratch'
shapefiledir = '/home/idies/workspace/Storage/raddick/Baltimore/shapefiles/'
tiger_dir = '/home/idies/workspace/Temporary/raddick/census_scratch/geography/TRACT_SHAPEFILES/'

thisdir = '/home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act/'
datadir = thisdir + 'datasets/'
figdir = thisdir + 'figures/'

os.chdir(thisdir)
os.getcwd()

Importing packages...


'/home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act'

In [2]:
# read aggregate data
agg_tracts_df = pandas.read_csv(datadir+'loans_by_census_tract_2010_2016.csv', encoding='utf-8', low_memory=False)
agg_tracts_df = agg_tracts_df[agg_tracts_df['activity_year'] >= 2012]

for thiscol in ['amtLoans1', 'amtLoans100k', 'amtLoans250k', 'amtLoansToSmallest', 'amtLoansTotal']:
    agg_tracts_df.loc[:, thiscol] = pandas.to_numeric(agg_tracts_df[thiscol].apply(lambda x: str(x).replace(',','')[1:]), errors='coerce')

# add shapefiles
tract_shapes_gdf = geopandas.read_file(tiger_dir +'tl_2017_24_tract.shp')
tract_shapes_gdf = tract_shapes_gdf[tract_shapes_gdf['COUNTYFP'] == '510']
tract_shapes_gdf.loc[:, 'NAME'] = pandas.to_numeric(tract_shapes_gdf['NAME'], errors='coerce')
agg_tracts_df = agg_tracts_df.merge(tract_shapes_gdf, how='left', left_on='census_tract', right_on='NAME')

# Get community statistical area names
tract_to_csa_df = pandas.read_csv('census_tract_to_neighborhood.csv', index_col='NAME10')
agg_tracts_df = agg_tracts_df.assign(CSA2010 = agg_tracts_df.join(tract_to_csa_df, how='left', on='census_tract')['CSA2010'])

# Get income group names
agg_tracts_df = agg_tracts_df.assign(income_group = np.nan)
agg_tracts_df.loc[agg_tracts_df['income_group_code'] == 1, 'income_group'] = '< 10% of Median Family Income (MFI)'
agg_tracts_df.loc[agg_tracts_df['income_group_code'] == 2, 'income_group'] = '10% to 20% of MFI'
agg_tracts_df.loc[agg_tracts_df['income_group_code'] == 3, 'income_group'] = '20% to 30% of MFI'
agg_tracts_df.loc[agg_tracts_df['income_group_code'] == 4, 'income_group'] = '30% to 40% of MFI'
agg_tracts_df.loc[agg_tracts_df['income_group_code'] == 5, 'income_group'] = '40% to 50% of MFI'
agg_tracts_df.loc[agg_tracts_df['income_group_code'] == 6, 'income_group'] = '50% to 60% of MFI'
agg_tracts_df.loc[agg_tracts_df['income_group_code'] == 7, 'income_group'] = '60% to 70% of MFI'
agg_tracts_df.loc[agg_tracts_df['income_group_code'] == 8, 'income_group'] = '70% to 80% of MFI'
agg_tracts_df.loc[agg_tracts_df['income_group_code'] == 9, 'income_group'] = '80% to 90% of MFI'
agg_tracts_df.loc[agg_tracts_df['income_group_code'] == 10, 'income_group'] = '90% to 100% of MFI'
agg_tracts_df.loc[agg_tracts_df['income_group_code'] == 11, 'income_group'] = '100% to 110% of MFI'
agg_tracts_df.loc[agg_tracts_df['income_group_code'] == 12, 'income_group'] = '110% to 120% of MFI'
agg_tracts_df.loc[agg_tracts_df['income_group_code'] == 13, 'income_group'] = '> 120% of MFI'

agg_tracts_gdf = geopandas.GeoDataFrame(agg_tracts_df, crs=tract_shapes_gdf.crs, geometry='geometry')
#agg_tracts_gdf = agg_tracts_gdf.set_index(['census_tract', 'activity_year'])
agg_tracts_2015_gdf = agg_tracts_gdf[agg_tracts_gdf['activity_year'] == 2015]
agg_tracts_2015_gdf = agg_tracts_2015_gdf.set_index('census_tract')

#agg_tracts_gdf[agg_tracts_gdf.index.get_level_values('activity_year') == 2016].plot(column='nLoans1')
#plt.show()
#agg_tracts_gdf['census_tract'][(agg_tracts_gdf['activity_year'] == 2016)].drop_duplicates()#[:40] #& (agg_tracts_gdf['census_tract'].apply(lambda x: int(str(x)[0]) > 2))]
#--101.00, 201.00 801.01, 801.02, 1001.00, 1308.03, 2001.00, 2101.00, 2501.01, 2606.01, 2701.01, 2801.01
print('backing up...')
agg_tracts_2015_gdf_bk = agg_tracts_2015_gdf
print('Done')
agg_tracts_2015_gdf.head(20)

backing up...
Done


,activity_year,income_group_code,nLoans1,amtLoans1,nLoans100k,amtLoans100k,nLoans250k,amtLoans250k,nLoansToSmallest,amtLoansToSmallest,...,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,CSA2010,income_group
census_tract,,,,,,,,,,,,,,,,,,,,,
101.0,2015,10,67,907000,0,0,4,2266000,33,1203000,...,Census Tract 101,G5020,S,393198,148387,+39.2786343,-076.5745793,"POLYGON ((-76.57614699999999 39.276733, -76.57...",Canton,90% to 100% of MFI
102.0,2015,9,26,258000,0,0,1,350000,18,155000,...,Census Tract 102,G5020,S,355165,0,+39.2880851,-076.5740101,"POLYGON ((-76.577242 39.286242, -76.576514 39....",Patterson Park North & East,80% to 90% of MFI
103.0,2015,13,41,500000,1,200000,0,0,16,147000,...,Census Tract 103,G5020,S,673883,10498,+39.2883650,-076.5802265,"POLYGON ((-76.584722 39.291891, -76.583983 39....",Canton,> 120% of MFI
104.0,2015,13,113,1766000,1,250000,11,6534000,63,5351000,...,Census Tract 104,G5020,S,373743,558487,+39.2785000,-076.5797059,"POLYGON ((-76.58705599999999 39.285853, -76.58...",Canton,> 120% of MFI
105.0,2015,13,15,195000,1,250000,0,0,11,154000,...,Census Tract 105,G5020,S,155244,0,+39.2888746,-076.5858889,"POLYGON ((-76.587414 39.291813, -76.58684 39.2...",Fells Point,> 120% of MFI
201.0,2015,10,30,413000,0,0,0,0,19,303000,...,Census Tract 201,G5020,S,174535,0,+39.2887712,-076.5887716,"POLYGON ((-76.590468 39.29167899999999, -76.58...",Fells Point,90% to 100% of MFI
202.0,2015,7,40,373000,0,0,0,0,30,293000,...,Census Tract 202,G5020,S,197559,0,+39.2886357,-076.5920299,"POLYGON ((-76.59394399999999 39.291542, -76.59...",Fells Point,60% to 70% of MFI
203.0,2015,12,177,2320000,3,550000,9,4897000,103,2707000,...,Census Tract 203,G5020,S,662775,384066,+39.2808610,-076.5940789,"POLYGON ((-76.602487 39.279571, -76.6005129999...",Fells Point,110% to 120% of MFI
301.0,2015,3,48,361000,2,271000,1,950000,27,1166000,...,Census Tract 301,G5020,S,427357,0,+39.2894071,-076.5965747,"POLYGON ((-76.599684 39.292526, -76.5991459999...",Harbor East/Little Italy,20% to 30% of MFI


# Add number of jobs per census tract

In [3]:
print('getting from backup...')
agg_tracts_2015_gdf = agg_tracts_2015_gdf_bk

print('reading job data...')
#agg_tracts_2016_acs5_2017.to
jobs_df = pandas.read_csv(datadir+'md_wac_S000_JT02_2010_to_2015.csv')
jobs_df.columns = [x.strip() for x in jobs_df.columns.tolist()]

# remove the starting "24510" from all geographies
jobs_df = jobs_df.assign(census_code = jobs_df['Workplace Census Block Code'].apply(lambda x: str(x)[5:])).astype('str')

jobs_df = jobs_df.assign(census_tract = np.nan)
# Take care of 1001, 1002, 1003
jobs_df.loc[jobs_df['census_tract'].isnull() & jobs_df['census_code'].apply(lambda x: x[0:3] == '100'), 'census_tract'] = pandas.to_numeric(jobs_df['census_code'][jobs_df['census_tract'].isnull() & jobs_df['census_code'].apply(lambda x: x[0:3] == '100')].apply(lambda x: x[0:4]+'.'+x[4:6]), errors='coerce')
# 1101 to 1901
jobs_df.loc[jobs_df['census_tract'].isnull() & jobs_df['census_code'].apply(lambda x: (int(x[0:2]) >= 11) & (int(x[0:2]) <= 19)), 'census_tract'] = pandas.to_numeric(jobs_df['census_code'][jobs_df['census_tract'].isnull() & jobs_df['census_code'].apply(lambda x: (int(x[0:2]) >= 11) & (int(x[0:2]) <= 19))].apply(lambda x: x[0:4]+'.'+x[4:6]), errors='coerce')
# 2101 to 2805
jobs_df.loc[jobs_df['census_tract'].isnull() & jobs_df['census_code'].apply(lambda x: (int(x[0:2]) >= 21) & (int(x[0:2]) <= 28)), 'census_tract'] = pandas.to_numeric(jobs_df['census_code'][jobs_df['census_tract'].isnull() & jobs_df['census_code'].apply(lambda x: (int(x[0:2]) >= 21) & (int(x[0:2]) <= 28))].apply(lambda x: x[0:4]+'.'+x[4:6]), errors='coerce')
# 201, 202
jobs_df.loc[jobs_df['census_tract'].isnull() & jobs_df['census_code'].apply(lambda x: x[0:4] in ('0201', '0202')), 'census_tract'] = pandas.to_numeric(jobs_df['census_code'][jobs_df['census_tract'].isnull() & jobs_df['census_code'].apply(lambda x: x[0:4] in ('0201', '0202'))].apply(lambda x: x[0:4]+'.'+x[4:6]), errors='coerce')
# 2001 to 2008
jobs_df.loc[jobs_df['census_tract'].isnull() & jobs_df['census_code'].apply(lambda x: x[0:3] == '200'), 'census_tract'] = pandas.to_numeric(jobs_df['census_code'][jobs_df['census_tract'].isnull() & jobs_df['census_code'].apply(lambda x: x[0:3] == '200')].apply(lambda x: x[0:4]+'.'+x[4:6]), errors='coerce')
# the rest
jobs_df.loc[jobs_df['census_tract'].isnull(), 'census_tract'] = pandas.to_numeric(jobs_df['census_code'][jobs_df['census_tract'].isnull()].apply(lambda x: x[0:4]+'.'+x[4:6]), errors='coerce')

sumcols = jobs_df.columns[1:-3].tolist()

for thiscol in sumcols:
    jobs_df[thiscol] = pandas.to_numeric(jobs_df[thiscol], errors='coerce')

print('getting total jobs by census tract in 2015...')
jobs_2015_df = jobs_df[jobs_df['Year'] == '2015']
jobs_2015_by_tract_df = pandas.DataFrame(data=None, columns=sumcols, index=jobs_2015_df['census_tract'].drop_duplicates().values.tolist())

print('Finding job variables by census tract...')
i = 0
for ix in jobs_2015_by_tract_df.index:
    if ((np.mod(i, 50) == 0) | (i == len(jobs_2015_by_tract_df.index)-1)):
        print('Processing census tract {0:,.0f} of {1:,.0f}...'.format(i+1, len(jobs_2015_by_tract_df.index)))
    for thiscol in sumcols:
        jobs_2015_by_tract_df.loc[ix, thiscol] = jobs_2015_df[thiscol][jobs_2015_df['census_tract'] == ix].sum()
    i = i + 1

jobs_2015_by_tract_df.index.name = 'census_tract'

print('joining 2015 job data onto 2015 loan data...')
agg_tracts_2015_gdf = agg_tracts_2015_gdf.join(jobs_2015_by_tract_df, how='left')

#print('calculating loans per job...')
agg_tracts_2015_gdf = agg_tracts_2015_gdf.assign(nLoans_per_job = agg_tracts_2015_gdf['nLoansTotal'] / agg_tracts_2015_gdf['Total number of jobs'])
agg_tracts_2015_gdf = agg_tracts_2015_gdf.assign(amtLoans_per_job = agg_tracts_2015_gdf['amtLoansTotal'] / agg_tracts_2015_gdf['Total number of jobs'])

#agg_tracts_2015_gdf.head(20)

print('backing up...')
agg_tracts_2015_gdf_bk = agg_tracts_2015_gdf

print('Done!')


getting from backup...
reading job data...
getting total jobs by census tract in 2015...
Finding job variables by census tract...
Processing census tract 1 of 200...
Processing census tract 51 of 200...
Processing census tract 101 of 200...
Processing census tract 151 of 200...
Processing census tract 200 of 200...
joining 2015 job data onto 2015 loan data...
backing up...
Done!


In [4]:
print('getting from backup...')
agg_tracts_2015_gdf = agg_tracts_2015_gdf_bk

print('getting ACS 5-year estimates for 2012-2017...')
acs5_2017_df = pandas.read_csv(datadir+'acs5/cra_iv_estimates.csv', low_memory=False, index_col='tract')

acs5_2017_df.index.name = 'census_tract'

agg_tracts_2015_gdf = agg_tracts_2015_gdf.join(acs5_2017_df)

print('writing outfile...')
agg_tracts_2015_gdf.to_csv(datadir+'cra_data_final_for_mac_2015.csv')
print('Done!')


getting from backup...
getting ACS 5-year estimates for 2012-2017...
writing outfile...
Done!


In [ ]:
# Unit of analysis: census tract
# DV: nLoans1, amtLoans1, etc. Also: (overall minus "credit card loans")
#### where credit card loans" are <$100k to businesses with <$10k budget
# IV: [population by race; owner-occupied units; MFI; hs grad pct; 
### female hoh pct; unemployment pct; poverty pct; median home value;
### median home year built